In [2]:
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", None)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, countDistinct
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/10 19:10:27 WARN Utils: Your hostname, LappyToppy, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/10 19:10:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/10 19:10:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
bronze_path = "/home/gnana/football-lakehouse/bronze"
silver_path = "/home/gnana/football-lakehouse/silver"

In [5]:
players = spark.read.parquet(f"{bronze_path}/players")
players.printSchema()

root
 |-- player_id: integer (nullable = true)
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- name: string (nullable = true)
 |-- last_season: integer (nullable = true)
 |-- current_club_id: integer (nullable = true)
 |-- player_code: string (nullable = true)
 |-- country_of_birth: string (nullable = true)
 |-- city_of_birth: string (nullable = true)
 |-- country_of_citizenship: string (nullable = true)
 |-- date_of_birth: timestamp (nullable = true)
 |-- sub_position: string (nullable = true)
 |-- position: string (nullable = true)
 |-- foot: string (nullable = true)
 |-- height_in_cm: integer (nullable = true)
 |-- contract_expiration_date: timestamp (nullable = true)
 |-- agent_name: string (nullable = true)
 |-- image_url: string (nullable = true)
 |-- url: string (nullable = true)
 |-- current_club_domestic_competition_id: string (nullable = true)
 |-- current_club_name: string (nullable = true)
 |-- market_value_in_eur: integer (nullable =

In [6]:
players.count()

32601

In [7]:
players.limit(5).toPandas()

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,1978-06-09,Centre-Forward,Attack,right,184.0,NaT,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrait/header/10-1448468291.jpg?lm=1,https://www.transfermarkt.co.uk/miroslav-klose/profil/spieler/10,IT1,Società Sportiva Lazio S.p.A.,1000000,30000000
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,1980-08-06,Goalkeeper,Goalkeeper,left,190.0,NaT,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrait/header/26-1502448725.jpg?lm=1,https://www.transfermarkt.co.uk/roman-weidenfeller/profil/spieler/26,L1,Borussia Dortmund,750000,8000000
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,1981-01-30,Centre-Forward,Attack,NaN,NaN,NaT,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrait/header/65-1683670068.jpg?lm=1,https://www.transfermarkt.co.uk/dimitar-berbatov/profil/spieler/65,GR1,Panthessalonikios Athlitikos Omilos Konstantinoupoliton,1000000,34500000
3,77,NaN,Lúcio,Lúcio,2012,506,lucio,Brazil,Brasília,Brazil,1978-05-08,Centre-Back,Defender,NaN,NaN,NaT,NaN,https://img.a.transfermarkt.technology/portrait/header/77-1458201664.jpg?lm=1,https://www.transfermarkt.co.uk/lucio/profil/spieler/77,IT1,Juventus Football Club,200000,24500000
4,80,Tom,Starke,Tom Starke,2017,27,tom-starke,East Germany (GDR),Freital,Germany,1981-03-18,Goalkeeper,Goalkeeper,right,194.0,NaT,IFM,https://img.a.transfermarkt.technology/portrait/header/80-1471439914.JPG?lm=1,https://www.transfermarkt.co.uk/tom-starke/profil/spieler/80,L1,FC Bayern München,100000,3000000


In [8]:
players_silver = players.select(
    col("player_id"),
    trim(col("name")).alias("name"),
    col("current_club_id"),
    col("position"),
    col("sub_position"),
    col("date_of_birth"),
    col("country_of_birth").alias("birth_country"),
    col("city_of_birth").alias("birth_city"),
    col("country_of_citizenship"),
    col("foot"),
    col("height_in_cm"),
    col("market_value_in_eur")
).dropDuplicates(["player_id"])

In [25]:
players_silver.limit(5).toPandas()

,player_id,name,current_club_id,position,sub_position,date_of_birth,birth_country,birth_city,country_of_citizenship,foot,height_in_cm,market_value_in_eur
0,26,Roman Weidenfeller,16,Goalkeeper,Goalkeeper,1980-08-06,Germany,Diez,Germany,left,190.0,750000
1,65,Dimitar Berbatov,1091,Attack,Centre-Forward,1981-01-30,Bulgaria,Blagoevgrad,Bulgaria,NaN,NaN,1000000
2,132,Tomas Rosicky,11,Midfield,Attacking Midfield,1980-10-04,CSSR,Praha,Czech Republic,both,179.0,350000
3,325,Tomas Ujfalusi,141,Defender,Centre-Back,1978-03-24,CSSR,Rýmařov,Czech Republic,NaN,NaN,300000
4,332,Benjamin Köhler,24,Midfield,Left Midfield,1980-08-04,Germany,Berlin,Germany,NaN,NaN,150000


In [9]:
players_silver.select(countDistinct('player_id')).show()

+-------------------------+
|count(DISTINCT player_id)|
+-------------------------+
|                    32601|
+-------------------------+



In [10]:
players_silver.write.mode("overwrite").parquet(f"{silver_path}/players_silver")